<a href="https://colab.research.google.com/github/BenGutteridge/4YP/blob/master/GMM_var_inf_Bishop/Variational_GMM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook is based on Chapter 9 (?) of Bishop, and aims to replicate the 'variational mixture of Gaussians' example worked through there. This is to help my understanding of this example, as much of it is the basis for the gradient ascent approach used in my 4YP, and to hopefully get functioning posterior inference. I'm stuck in a rut, and better to go from a working model than try and debug a crummy bloated one built from scratch that has never worked!

# To do
- combine the crap together into a utils file
- clean up explanation
- get the code nice and together
- get rid of the sample stuff for the moment

to clear up:
- why use Sk and m for plots?
- why do we need to sample r from a dirichlet to start with?

# Intro
Distribution of $Z$:

i.e. the assignment $z_i$ for a single $x_i$ equals $k$ with probability $\pi_k$, where $\sum \limits^K \pi_k = 1$

$p(Z|\pi) = \prod \limits^N \prod \limits^K \pi_{k}^{z_{nk}}$

Conditional distribution of observed data (i.e. dataset given true labels and Gaussian means/precisions):

$p(X|Z,\mu,\Lambda) = \prod \limits^N \prod \limits^K \mathcal{N}(x_n | \mu_k, \Lambda_k^{-1})^{z_{nk}}$




#### The joint distribution

Factorises as follows:

$p(X,Z,\pi,\mu,\Lambda) = p(X|Z,\mu,\Lambda)p(Z|\pi)p(\pi)p(\mu|\Lambda)p(\Lambda)$


#### The variational distribution

Takes the following form - the only assumption is that it factorises between latent variablesd $Z$ and the parameters:

$q(Z,\pi,\mu,\Lambda) = q(Z)q(\pi,\mu,\Lambda)$

The fundamental general expression for the iterative variaitional approach here is:

$\log q_j^*(Z_j) = \mathbb{E}_{i \neq j}[\log p(X,Z)] + c$

i.e. look at the expectation of the joint of $\log p$ for all the latent variables except the one we are predicting - in our case for considering just $Z$ this is:

$\log q^*(Z) = \mathbb{E}_{\pi,\mu,\Lambda}[\log p(X,Z,\pi,\mu,\Lambda)] + c$

(N.b. when we write $\mathbb{E}_{\pi,\mu,\Lambda}$, what this really is is $\mathbb{E}_{q(\pi,\mu,\Lambda)}$)

And then decompose the joint into $\mathbb{E}_\pi[\log p(Z|\pi)] + \mathbb{E}_{\mu,\Lambda}[\log p(X|Z,\mu,\Lambda)] + c$

Some reworking and taking the exponent again brings us to:

$q^*(Z) = \prod \limits^N \prod \limits^K r_{nk}^{z_{nk}}$
where $r_{nk} = \dfrac{\rho_{nk}}{\sum \limits^K_{j=1} \rho_{nj}}$ and $\rho_{nk} = \pi_k \mathcal{N}(x_n|\mu,\Lambda_k^{-1})$

i.e. we have derived that the form of the $Z$ factor of the variaitonal distribution takes the form of a multinomial, with a responsibility of each mixture component $k$ for each datapoint $n$


This still leaves the other factor of the variational distribution, $q(\pi,\mu,\Lambda)$. Again using the general idea of the expectation over the joint,

$\log q^*(\pi,\mu,\Lambda) = \mathbb{E}_Z[\log p(X,Z,\pi,\mu,\Lambda)] =\\ \mathbb{E}_Z[\log p(X|Z,\mu,\Lambda) + \log p(Z|\pi) + \log p(\pi) + \log p(\mu,\Lambda)]$

The last two terms simply equal themselves as they don't have a $Z$ component (can go outside of integral and the expectiation is simply an integral over a distribution summing to 1), but the first two have an expectation.

From choice of priors, $p(\pi)$ is a Dirichlet distribution $= C(\alpha_0)\prod \limits^K \pi_k^{\alpha_0-1}$

$p(\mu,\Lambda)$ is a factorised product of Gaussian-Wishart: $p(\mu|\Lambda)p(\Lambda) = \prod \limits^K \mathcal{N}(\mu_k|m_0,(\beta_0 \Lambda)^{-1})\mathcal{W}(\Lambda_k|W_0,\nu_0)$

#### Gaussian-Wishart parameters
https://math.stackexchange.com/questions/2803164/degrees-of-freedom-in-a-wishart-distribution

$m_0$ is the 'location' of the mean $\mu$ chosen usually as zero by symmetry

$\beta_0$: choosing a large $\beta_0$ tightens the Gaussian from which $\mu$ is sampled.

$W_0$ is the positive definite scale matrix

If we have some prior idea of the covariance matrix $\Lambda^{-1}$, $\Sigma_0$, we use $\nu$ and $W$ to encode our confidence in it. We set $W = \Sigma_0^{-1}$. The mean (expectation) of a Wishart is $\nu W$ and its mode is $(\nu-d-1) W$. the least informative prior is $\nu=d$, or possibly $d+2$ - something small. Choosing a large $\nu$ encodes confidence in the prior $\Sigma_0$. 

back to the variational distribution. There are clearly two parts, $q(\pi)$ and $q(\mu,\Lambda)$.

$q(\pi) = \log p(\pi) + \mathbb{E}_Z[\log p(Z|\pi)] = \log C(\alpha_0) + \log(\alpha_0 - 1) \sum \limits^K \pi_k + \sum \limits^{N,K} \mathbb{E}[z_{nk}] \log \pi_k$

$\mathbb{E}_Z[z_{nk}]$ is the expectation of a multinomial, i.e. number of trials*probability, $=r_{nk}$

$q^*(\pi) = C_1 \prod \limits^K \pi_k^{\alpha_0-1}\prod \limits^N \pi_k^{r_{nk}}= C_1\prod \limits^K \pi_k^{(\alpha_0-1+\sum \limits^N r_{nk})} = Dir(\pi|\alpha)$ where $\alpha_k = \alpha_0 + \sum \limits^N r_{nk}$

Cool!

And now for $\log q^*(\mu,\Lambda)$:

$\log q^*(\mu,\Lambda) = \sum \limits^K \log p(\mu_k,\Lambda_k) + \mathbb{E}_Z[\log p(X|Z,\mu,\Lambda)] = \\
\sum \limits^K \log (\mathcal{N}(\mu_k|m_0,\beta_0,\Lambda_k) \mathcal{W}(\Lambda_k|\nu,W_0)) + \sum \limits^{N,K} \mathbb{E}_Z[z_{nk}]\log \mathcal{N}(x_n|\mu_k,\Lambda_k^{-1})$

Similar to q*(\pi) this turns into a form of a Gaussian-Wishart with different parameters.

In [ ]:
import numpy as np
from numpy.linalg import inv, det, multi_dot
import scipy
import matplotlib.pyplot as plt
from utils import *

In [ ]:
def alpha_k(Nk, alpha0):
  return alpha0 + Nk

def beta_k(Nk, beta0):
  return beta0 + Nk

def m_k(Nk, xkbar, betak, m0, beta0):
  return ((1/betak)*(beta0*m0 + Nk*xkbar)).reshape(2)

def W_k(Nk, xkbar, Sk, m0, beta0):
  inv_Wk = inv(W0) + Nk*Sk + ((beta0*Nk)/(beta0+Nk))*np.dot((xkbar-m0),(xkbar-m0).T) 
  return inv(inv_Wk)

def nu_k(Nk, nu0):
  return nu0 + Nk

These use some new parameters, $N_k, \bar{x}_k, S_k$, which essentially correspond to the sum of responsibilities, responsibility-weighted mean and responsibility-weighted variance for mixture component $k$:

In [ ]:
def N_k(responsibilities_k):
  return np.sum(responsibilities_k)

def x_k_bar(Nk, responsibilities_k, X):
  N = responsibilities_k.shape[0]
  D = X.shape[1]
  sum = np.zeros((1,D))
  for n in range(N):
    sum = sum + responsibilities_k[n]*X[n]
  if Nk > 0:
    return (1/Nk)*sum
  else:
    return 0.

def S_k(Nk, responsibilities_k, X, xkbar):
  N = responsibilities_k.shape[0]
  sum = 0.0
  for n in range(N):
    sum = sum + responsibilities_k[n]*np.dot((X[n]-xkbar).T,(X[n]-xkbar))
  if Nk > 0:
    return (1/Nk)*sum
  else:
    return np.eye(2) # doesn't actually matter what it returns, the component is dead

These updates correspond to the M (maximisation) step of the EM algorithm. To perform them we of course need our responsibilities:

$\mathbb{E}_Z[r_{nk}] = \dfrac{\rho_{nk}}{\sum \limits^K_{j=1} \rho_{nj}}$ and
$\rho_{nk} = \pi_k \mathcal{N}(x_n|\mu,\Lambda_k^{-1})$

The $\rho_{nk}$ above is not quite right --- we have ignored the $\mathbb{E}_{\mu,\lambda}[.]$ Let's consider in full:

$\begin{aligned}
\ln \rho_{n k}=& \mathbb{E}\left[\ln \pi_{k}\right]+\frac{1}{2} \mathbb{E}\left[\ln \left|\boldsymbol{\Lambda}_{k}\right|\right]-\frac{D}{2} \ln (2 \pi) \\
&-\frac{1}{2} \mathbb{E}_{\mu_{k}, \Lambda_{k}}\left[\left(\mathbf{x}_{n}-\boldsymbol{\mu}_{k}\right)^{\mathrm{T}} \boldsymbol{\Lambda}_{k}\left(\mathbf{x}_{n}-\boldsymbol{\mu}_{k}\right)\right]
\end{aligned}$

There are four components here:


1.   $\mathbb{E}[\ln \pi_{k}] = \psi(\alpha_k) - \psi(\sum \limits^K \alpha_k)$, using the digamma function $\psi(x) = \dfrac{d}{dx}\log \Gamma = \dfrac{\Gamma '}{\Gamma}$. This is the known log-expectation of the Dirichlet distribution.
2.   $ \mathbb{E}\left[\ln \left|\boldsymbol{\Lambda}_{k}\right|\right] = \sum_{i=1}^{D} \psi\left(\frac{\nu_{k}+1-i}{2}\right)+D \ln 2+\ln \left|\mathbf{W}_{k}\right|$. This is the known log-expectation of the Wishart distribution.
3. Easy
4. $\mathbb{E}_{\boldsymbol{\mu}_{k}, \boldsymbol{\Lambda}_{k}}\left[\left(\mathbf{x}_{n}-\boldsymbol{\mu}_{k}\right)^{\mathrm{T}} \boldsymbol{\Lambda}_{k}\left(\mathbf{x}_{n}-\boldsymbol{\mu}_{k}\right)\right] = D\beta_{k}^{-1}+\nu_{k}\left(\mathbf{x}_{n}-\mathbf{m}_{k}\right)^{\mathrm{T}} \mathbf{W}_{k}\left(\mathbf{x}_{n}-\mathbf{m}_{k}\right)$. The proof of this is fiddly but it does work (see https://github.com/zhengqigao/PRML-Solution-Manual, p209).


In [ ]:
from scipy.special import digamma
def ln_rho_nk(k, alpha, nu, W, beta, m, xn):
  D = X.shape[1] # dimensionality
  
  E_ln_pi_k =  digamma(alpha[k]) - digamma(np.sum(alpha))
  E_ln_lam_k = np.sum(digamma(nu[k]+1-np.arange(D)+1)) + D*np.log(2) + np.log(det(W[k]))
  E_ln_mu_k = D*beta[k]**-1 + nu[k]*multi_dot(((xn-m[k]).T, W[k], (xn-m[k])))
  
  return E_ln_pi_k + 0.5*E_ln_lam_k - 0.5*D*np.log(2*np.pi) - 0.5*E_ln_mu_k

Now we need $r_{nk}$, which we remember is $\rho_{nk}$ divided by the sum of $\rho_{nk}$ over $k$:

In [ ]:
def r_nk(k, alpha, nu, W, beta, m, xn):
  rhonk = np.exp(ln_rho_nk(k, alpha, nu, W, beta, m, xn))
  sum_k_rho = 0.
  for j in range(K):
    sum_k_rho += np.exp(ln_rho_nk(j, alpha, nu, W, beta, m, xn))
  return rhonk/sum_k_rho

### The algorithm
E step: Calculate responsibilities $r_{nk}$ for each and every mixture component $k$ and datapoint $x_n$.

M step: use $r_{nk}$ in the update equations for $\alpha, \beta, m, W, \nu$ to minimise the $\mathbb{KL}$ divergence between the variational distribution $q(Z,\pi,\mu,\Lambda)$ and the true posterior $p(Z,\pi,\mu,\Lambda|X)$

In [ ]:
# E step: calculate responsibility
def E_step(N,K,alpha,nu,W,beta,m,X):
  r = np.empty((N,K))
  for n in range(N):
    for k in range(K):
      r[n,k] = r_nk(k, alpha, nu, W, beta, m, X[n])
  return r

In [ ]:
# M step: update hyperparameters
def M_step(r,X,alpha0,beta0,m0,W0,nu0):
  NK, xbar, S = [],[],[]
  alpha, beta, nu = np.empty(K), np.empty(K), np.empty(K)
  m, W = [np.zeros(2) for _ in range(K)], [np.zeros((2,2)) for _ in range(K)]

  for k in range(K):
    Nk = N_k(r[:,k])
    print('N_%d = '%k, Nk)
    xkbar = x_k_bar(Nk, r[:,k], X)
    Sk = S_k(Nk, r[:,k], X, xkbar)

    alpha[k] = alpha_k(Nk, alpha0)
    beta[k] = beta_k(Nk, beta0)
    m[k] = m_k(Nk, xkbar, beta[k], m0, beta0)
    W[k] = W_k(Nk, xkbar, Sk, m0, beta0)
    nu[k] = nu_k(Nk, nu0)

    NK.append(Nk)
    xbar.append(xkbar)
    S.append(Sk)

    # print('k=%d\nalpha'%k, alpha[k], '\nbeta', beta[k], '\nm', m[k], '\nW', W[k], '\nnu', nu[k])
  return alpha, beta, m, W, nu, NK, xbar, S

# Code


1.   Choose some priors, $\mathbf{\theta}_0$, set $\mathbf{\theta} = \mathbf{\theta}_0$ 
2.   Iterate until converged:   
  1.   Calculate the full set of $NK$ responsibilities
  2.   Using update equations, update each $\theta_k$

In [ ]:
from numpy.random import multivariate_normal
import matplotlib.pyplot as plt

# Set priors, dataset, problem
D = 2       # dimensions
K = 5       # number of components, unnecessary ones should go to zero
N = 500     # number of points in synthetic dataset
N_its = 100 # number of updates

# Dataset
centres = [np.array([0.,8.]), np.array([5.,0.])]
covs = [np.eye(2), np.array([[0.6,0.4],
                             [0.4,0.6]])] 
X1 = multivariate_normal(mean=centres[0],
                         cov=covs[0],
                         size=int(N/2))
X2 = multivariate_normal(mean=centres[1],
                         cov=covs[1],
                         size=int(N/2))
X = np.concatenate((X1,X2))

In [ ]:
# Variational priors
alpha0 = 1e-3     # as alpha0 -> 0, pi_k -> 0. As alpha0 -> Inf, pi_k -> 1/K
beta0 = 1e-10     # ???
m0 = np.zeros(2)  # zero by convention (symmetry)
W0 = np.eye(2)    # 
nu0 = 2           # 

In [ ]:
%matplotlib notebook 
# hacky, but this suppresses plots

# r needs to be randomised (not uniform) because if its all the same nothing 
# changes - not sure of the mathematical reasoning for this
r = np.array([np.random.dirichlet(np.ones(K)) for _ in range(N)])

# neaten up the output
from tqdm.notebook import tqdm
import warnings
warnings.simplefilter('ignore')

with HiddenPrints():
# for _ in range(1):
  for i in tqdm(range(N_its)):
    print('\n******************Iteration %d************************\n'%i)
    
    # M step
    alpha, beta, m, W, nu, NK, xbar, S = M_step(r,X,alpha0,beta0,m0,W0,nu0)
    print('\nalpha', alpha, '\nbeta', beta, '\nnu', nu, '\nm', m, '\nW', W, '\nnu', nu)

    # mu, lam = [],[]
    # pi = sample_pi(alpha)
    # for k in range(K):
    #   lam.append(sample_lambda(W[k], nu[k]))
    #   mu.append(sample_mu(m[k], beta[k], lam[k]))

    def E_pi(alpha, alpha0, N):
      return [(alpha[k])/(K*alpha0 + N) for k in range(K)]
    
    Epi = E_pi(alpha, alpha0, N)
    print('E[pi] = ', Epi)

    # Plot
    title = 'iteration %d' % i
    filename = 'plots/img%04d.png'%i
    # plot_GMM(X, mu, lam, pi, centres, covs, K, title)
    plot_GMM(X, m, inv(S), Epi, centres, covs, K, title, savefigpath=filename)

    # E step
    r = E_step(N,K,alpha,nu,W,beta,m,X)

In [ ]:
%matplotlib inline
# Make and display gif 
filedir = 'plots'
gifname = make_gif(filedir)

# delete pngs for next run
for file in os.listdir(filedir):
  os.remove(os.path.join(filedir,file))

Image(open("/content/%s.gif"%gifname,'rb').read())



## Calculating ELBO
Now lets get the programme to calculate the evidence lower bound on each iteration. From Bishop, eq.10.70:

\begin{aligned}
\mathcal{L}=& \sum_{\mathbb{Z}} \iiint q(\mathbf{Z}, \boldsymbol{\pi}, \boldsymbol{\mu}, \boldsymbol{\Lambda}) \ln \left\{\frac{p(\mathbf{X}, \mathbf{Z}, \boldsymbol{\pi}, \boldsymbol{\mu}, \boldsymbol{\Lambda})}{q(\mathbf{Z}, \boldsymbol{\pi}, \boldsymbol{\mu}, \boldsymbol{\Lambda})}\right\} \mathrm{d} \boldsymbol{\pi} \mathrm{d} \boldsymbol{\mu} \mathrm{d} \boldsymbol{\Lambda} \\
=& \mathbb{E}[\ln p(\mathbf{X}, \mathbf{Z}, \boldsymbol{\pi}, \boldsymbol{\mu}, \boldsymbol{\Lambda})]-\mathbb{E}[\ln q(\mathbf{Z}, \boldsymbol{\pi}, \boldsymbol{\mu}, \boldsymbol{\Lambda})] \\
=& \mathbb{E}[\ln p(\mathbf{X} \mid \mathbf{Z}, \boldsymbol{\mu}, \boldsymbol{\Lambda})]+\mathbb{E}[\ln p(\mathbf{Z} \mid \boldsymbol{\pi})]+\mathbb{E}[\ln p(\boldsymbol{\pi})]+\mathbb{E}[\ln p(\boldsymbol{\mu}, \boldsymbol{\Lambda})] \\
&-\mathbb{E}[\ln q(\mathbf{Z})]-\mathbb{E}[\ln q(\boldsymbol{\pi})]-\mathbb{E}[\ln q(\boldsymbol{\mu}, \boldsymbol{\Lambda})]
\end{aligned}